In [ ]:
!pip install rebound
import rebound
import numpy as np
import matplotlib.pyplot as plt

# Crear simulación
sim = rebound.Simulation()
sim.units = ("AU", "day", "Msun")


print("Bienvenido al problema de 3 cuerpos restringido de Euler")

# Definir las masas de los cuerpos
masa_sol = float(input("Ingrese la masa del Sol (en masas solares, caso realista 1): "))
masa_jupiter = float(input("Ingrese la masa de Júpiter (en masas solares, caso realista 0.001): "))
masa_particula_1 = float(input("Ingrese la masa de la primera partícula (0 para masa nula): "))
masa_particula_2 = float(input("Ingrese la masa de la segunda partícula (0 para masa nula): "))

# Ingresar distancia del Sol a Júpiter
r_jupiter = float(input("Ingrese la distancia del Sol a Júpiter (en UA, caso realista 5.2): "))

# Ingresar radios para las partículas
r_particula_1 = float(input("Ingrese la distancia del Sol a la primera partícula (en UA,caso realista 5.2): "))
r_particula_2 = float(input("Ingrese la distancia del Sol a la segunda partícula (en UA, caso realista 5.2): "))

tiempo_total = int(input("Ingrese el tiempo total de la simulación (en días, caso realista 4333 para una orbita de Júpiter): "))

# Añadir el Sol y Júpiter con las masas y distancia especificadas
sim.add(m=masa_sol)  # Sol
sim.add(m=masa_jupiter, a=r_jupiter, e=0)

# Colocar las partículas en L4 y L5
theta_particula_1 = np.radians(60)
theta_particula_2 = np.radians(-60)

# Añadir la primera partícula en L4
sim.add(m=masa_particula_1, a=r_particula_1, e=0, inc=0, Omega=0, omega=theta_particula_1)

# Añadir la segunda partícula en L5
sim.add(m=masa_particula_2, a=r_particula_2, e=0, inc=0, Omega=0, omega=theta_particula_2)

# Configurar el integrador
sim.integrator = "ias15"
sim.dt = 0.1

# Listas para guardar las posiciones
particula_1_x = []
particula_1_y = []
particula_2_x = []
particula_2_y = []
jupiter_x = []
jupiter_y = []
L4_x = []
L4_y = []
L5_x = []
L5_y = []
sim.move_to_com()

# Integrar y guardar posiciones
for t in np.arange(0, tiempo_total, sim.dt):
    sim.integrate(t)

    # Obtener posiciones
    jupiter = sim.particles[1]
    particula_1 = sim.particles[2]
    particula_2 = sim.particles[3]

    jupiter_x.append(jupiter.x)
    jupiter_y.append(jupiter.y)

    particula_1_x.append(particula_1.x)
    particula_1_y.append(particula_1.y)

    particula_2_x.append(particula_2.x)
    particula_2_y.append(particula_2.y)

    # Calcular las posiciones de L4 y L5
    theta_jupiter = np.arctan2(jupiter.y, jupiter.x)  # Ángulo de Júpiter
    L4_angle = theta_jupiter + np.radians(60)  # Desfase de 60 grados
    L5_angle = theta_jupiter - np.radians(60)  # Desfase de -60 grados

    L4_x.append(r_jupiter * np.cos(L4_angle))
    L4_y.append(r_jupiter * np.sin(L4_angle))

    L5_x.append(r_jupiter * np.cos(L5_angle))
    L5_y.append(r_jupiter * np.sin(L5_angle))

# Graficar resultados
plt.figure(figsize=(15, 10))
plt.plot(jupiter_x, jupiter_y, label='Órbita de Júpiter', color='red')
plt.plot(particula_1_x, particula_1_y, label='Trayectoria de la Partícula 1 (L4)', color='blue')
plt.plot(particula_2_x, particula_2_y, label='Trayectoria de la Partícula 2 (L5)', color='purple')
plt.scatter(0, 0, color='yellow', s=300, label='Sol')

# Graficar puntos de Lagrange L4 y L5
plt.plot(L4_x, L4_y, color='pink', label='L4', linestyle='--')
plt.plot(L5_x, L5_y, color='orange', label='L5', linestyle='--')


plt.xlabel('Posición X (AU)')
plt.ylabel('Posición Y (AU)')
plt.title('Simulación de Júpiter y Partículas en Puntos de Lagrange')
plt.axis('equal')
plt.grid(0)
plt.legend()
plt.show()